In [ ]:
""" import libraries """

import pandas as pd
import sqlite3 as sql
import sqlalchemy as alchemy
import datetime


""" read dataset from text file"""
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [211002, 210925, 210918,210911,210904,210828,210821,210814,210807,210731,210724,210717,210710,210703]
turnstiles_df = get_data(week_nums)

"""" make time series """

turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")

"""" add turnstile and weekday columns """

turnstiles_df['Turnstile'] = turnstiles_df['C/A'] + turnstiles_df['UNIT'] + turnstiles_df['SCP'] + turnstiles_df['STATION']
turnstiles_df['Week_Day'] = turnstiles_df['DATE_TIME'].dt.day_name()

"""" ingest data into DB """

engine = alchemy.create_engine('sqlite:///MTA.db')
turnstiles_df.to_sql('subway',engine, if_exists='append')